In [3]:
import pandas as pd
import numpy as np

factor_prices = pd.DataFrame(
    {
        "Factor 1": [100, 105, 110, 115, 120],
        "Factor 2": [200, 205, 210, 215, 220],
        "Factor 3": [300, 305, 310, 315, 320],
    },
    index=range(5),
)
factor_prices.head()

,Factor 1,Factor 2,Factor 3
0,100,200,300
1,105,205,305
2,110,210,310
3,115,215,315
4,120,220,320


In [5]:
log_returns = np.log(factor_prices / factor_prices.shift(1))
print(log_returns.dropna()) # type: ignore

   Factor 1  Factor 2  Factor 3
1  0.048790  0.024693  0.016529
2  0.046520  0.024098  0.016261
3  0.044452  0.023530  0.016000
4  0.042560  0.022990  0.015748


In [6]:
position_matrix = log_returns.T @ log_returns
position_matrix

,Factor 1,Factor 2,Factor 3
Factor 1,NaN,NaN,NaN
Factor 2,NaN,NaN,NaN
Factor 3,NaN,NaN,NaN


In [3]:
from dataclasses import dataclass
from functools import cached_property
from typing import Any

@dataclass
class PortfolioVarCalculator:
    '''
    class is responsible for calculation of different types
    of vars of elements that constiture the portfolio
    It has the following public methods:
        portfolio_var: returns portfolio var for given quantile
        isolated_var: returns isolated var for given quantile
        component_var: returns component var for given quantile
        incremental_var: returns all incremental vars for each asset

    Class attributes:
        prices_table: a dataframe that contains prices of all positions
        positions_table: a dataframe containing all
            positions of a given portfolio
    '''
    prices_table: pd.DataFrame
    positions_table: pd.DataFrame

    def __post_init__(self):
        # make sure that prices table consits only of factors
        # that are relevant for the given positions table
        var_tickers = self.positions_table['VaRTicker'].unique()
        self.prices_table = self.prices_table.loc[:, var_tickers] # type: ignore

    def portfolio_var(self, quantile: float) -> float:
        '''returns portfolio var for given quantile'''
        return self._portfolio_std * quantile

    def isolated_var(self, quantile: float) -> pd.Series:
        '''returns isolated var for given quantile'''
        return self._position_std * quantile

    def component_var(self, quantile: float) -> pd.Series:
        '''returns component var for given quantile'''
        return self._position_weights * self.portfolio_var(quantile)

    def incremental_var(self, quantile: float) -> pd.Series:
        '''returns all incremental vars for each asset'''

        return pd.Series(
            {
                ticker: self._ticker_incremental_var(ticker, quantile)
                for ticker in self._var_tickers
            }
        )

    @cached_property
    def _positions(self) -> pd.DataFrame:
        '''aggregated position exposure for each varTicker'''
        return self.positions_table\
            .groupby('VaRTicker')\
            .agg({'Exposure': 'sum'})

    @cached_property
    def _position_weights(self) -> pd.Series:
        '''returns position weights in the portfolio'''
        return self._positions.Exposure / self._positions.Exposure.sum()

    @cached_property
    def _position_returns(self) -> pd.DataFrame:
        '''returns daily returns of the portfolio'''
        return (
            (self.prices_table.shift(1) - self.prices_table)
            / self.prices_table.shift(1)
        )\
            .dropna()

    @cached_property
    def _return_covariance(self) -> pd.DataFrame:
        '''returns covariance matrix of the portfolio'''
        return self._position_returns.cov()

    @cached_property
    def _portfolio_std(self) -> float:
        '''returns standard deviation of the portfolio'''
        return np.sqrt(
            self._position_weights.T
            @ self._return_covariance
            @ self._position_weights
        )  # type: ignore

    @property
    def _var_tickers(self) -> pd.Series:
        '''returns var tickers'''
        return self._positions.index.to_series()

    @cached_property
    def _position_std(self) -> pd.Series:
        '''returns standard deviation of the portfolio positions'''
        return self._position_returns.std()

    @cached_property
    def _isolated_calculators(self) -> dict[str, Any]:
        '''returns isolated var calculators for each ticker'''

        return_dict = {}
        for ticker in self._var_tickers:
            incremental_positions = self.positions_table.loc[
                self.positions_table.VaRTicker != ticker, :
            ]
            return_dict[ticker] = PortfolioVarCalculator(
                prices_table=self.prices_table,
                positions_table=incremental_positions,
            )

        return return_dict

    def _ticker_incremental_var(
        self,
        ticker: str,
        quantile: float
    ) -> pd.Series:
        '''returns incremental var for a given ticker for given quantile'''

        return self.portfolio_var(quantile) \
            - self._isolated_calculators[ticker].portfolio_var(quantile)


In [4]:
import pandas as pd
import numpy as np
from scipy.stats import norm


# Define prices and positions data
prices_data = pd.DataFrame(
    {
        "Asset 1": [100, 106, 116, 123, 125, 133, 133, 124, 126, 121],
        "Asset 2": [200, 200, 200, 201, 199, 199, 196, 195, 195, 196],
        "Asset 3": [300, 310, 304, 311, 318, 315, 307, 315, 308, 311],
    },
    index=range(10),
)

positions_data = pd.DataFrame(
    {
        'VaRTicker': ['Asset 1', 'Asset 2', 'Asset 3', 'Asset 1'],
        'Exposure': [-300, 500, 400, -200],
        'Fund': ['Fund 1', 'Fund 1', 'Fund 2', 'Fund 2']
    }
)

quantile = norm.ppf(0.95)
portfolio_var_calculator = PortfolioVarCalculator(prices_data, positions_data)
portfolio_var = portfolio_var_calculator.portfolio_var(quantile=quantile)
print(f'{portfolio_var}')


log_returns = np.log(prices_data / prices_data.shift(1)).dropna() # type: ignore
return_covariance = log_returns.cov()

groupped_exposure = positions_data.groupby(['Fund', 'VaRTicker']).agg({'Exposure': 'sum'})
groupped_exposure['Weight'] = groupped_exposure.Exposure / groupped_exposure.Exposure.sum()
groupped_exposure.Weight

0.12099738250089814


Fund    VaRTicker
Fund 1  Asset 1     -0.75
        Asset 2      1.25
Fund 2  Asset 1     -0.50
        Asset 3      1.00
Name: Weight, dtype: float64

In [6]:
funds = groupped_exposure.index.levels[0]
fund_stds = pd.Series()
for fund in funds:
    fund_exposure = groupped_exposure.loc[fund].Weight
    fund_positions = list(fund_exposure.index)
    fund_covariance = return_covariance.loc[fund_positions, fund_positions]

    print(f'Covariance for fund: {fund}:\n{fund_covariance}')
    fund_std = (fund_exposure.T @ fund_covariance @ fund_exposure) ** .5
    print(f'Fund std: {fund_std:.6f}')
    fund_stds[fund] = fund_std


fund_weights = groupped_exposure.groupby(level=0).Weight.sum()
print(f'{fund_stds*quantile=}')
print(f'{fund_weights=}')

print(f'{portfolio_var * fund_weights}')


Covariance for fund: Fund 1:
          Asset 1   Asset 2
Asset 1  0.002729  0.000089
Asset 2  0.000089  0.000045
Fund std: 0.037940
Covariance for fund: Fund 2:
          Asset 1   Asset 3
Asset 1  0.002729 -0.000258
Asset 3 -0.000258  0.000542
Fund std: 0.038505
fund_stds*quantile=Fund 1    0.062406
Fund 2    0.063334
dtype: float64
fund_weights=Fund
Fund 1    0.5
Fund 2    0.5
Name: Weight, dtype: float64
Fund
Fund 1    0.060499
Fund 2    0.060499
Name: Weight, dtype: float64


In [10]:
print(groupped_exposure.index.levels[0])
print(groupped_exposure.index.levels[1])

Index(['Fund 1', 'Fund 2'], dtype='object', name='Fund')
Index(['Asset 1', 'Asset 2', 'Asset 3'], dtype='object', name='VaRTicker')


In [13]:
groupped_exposure.loc['Fund 1'].index.values

array(['Asset 1', 'Asset 2'], dtype=object)